In [ ]:
import numpy as np
import pandas as pd
import vector
import os
import hist
import awkward as ak

vector.register_awkward()

In [ ]:
import matplotlib.pyplot as plt
import matplotlib.ticker as mticker

# mplhep for CMS-style plots
import mplhep as hep

plt.style.use(hep.style.CMS)
hep.style.use("CMS")
formatter = mticker.ScalarFormatter(useMathText=True)
formatter.set_powerlimits((-3, 3))
plt.rcParams.update({"font.size": 16})

In [ ]:
def make_vector(events: pd.DataFrame, obj: str):
    """Create a ``vector`` object from the columns of the dataframe"""
    mstring = "Msd" if obj == "ak8FatJet" else "Mass"

    return vector.array(
        {
            "pt": events[f"{obj}Pt"],
            "phi": events[f"{obj}Phi"],
            "eta": events[f"{obj}Eta"],
            "M": events[f"{obj}{mstring}"],
        }
    )

In [ ]:
events = pd.read_parquet(
    "/eos/uscms/store/user/cmantill/bbbb/matching/Sep19/GluGlutoHHto4B_kl-1p00_kt-1p00_c2-0p00_TuneCP5_13p6TeV_TSG/parquet"
)
jets = make_vector(events, "ak4Jet")
gen_higgs = make_vector(events, "GenHiggs")
fjs = make_vector(events, "ak8FatJet")

In [ ]:
# ak4 jets matched to h1 and h2
h1ak4 = events["ak4JetHiggsMatchIndex"].to_numpy() == 0
h2ak4 = events["ak4JetHiggsMatchIndex"].to_numpy() == 1
num_ak4m2h1 = h1ak4.sum(axis=1)
num_ak4m2h2 = h2ak4.sum(axis=1)
h1m2ak4 = num_ak4m2h1 == 2
h2m2ak4 = num_ak4m2h2 == 2

# ak8 jets matched to h1 and h2
h1ak8 = events["ak8FatJetHiggsMatchIndex"].to_numpy() == 0
h2ak8 = events["ak8FatJetHiggsMatchIndex"].to_numpy() == 1
num_ak8m2h1 = h1ak8.sum(axis=1)
num_ak8m2h2 = h2ak8.sum(axis=1)
h1m1ak8 = num_ak8m2h1 == 1
h2m1ak8 = num_ak8m2h2 == 1

boosted = h1m1ak8 & h2m1ak8
resolved = (h1m2ak4 & h2m2ak4) & ~(boosted)
semi_resolved_h1 = h1m2ak4 & (h2m1ak8 & ~h2m2ak4) & ~(resolved) & ~(boosted)
semi_resolved_h2 = h2m2ak4 & (h1m1ak8 & ~h1m2ak4) & ~(resolved) & ~(boosted)
semi_resolved = semi_resolved_h1 | semi_resolved_h2
not_categorized = ~(resolved | boosted | semi_resolved)

## Classification from gen-level definition

We can classify events depending on matching into orthogonal categories:
- Boosted: 2 AK8 jets matched to H1 and H2, respectively
- Resolved: 4 AK4 jets matched to H1 and H2, 2 and 2, respectively - AND vetoing boosted
- Semi-resolved: "2 AK4 jets matched to H1/H2 and 1 AK8 jet matched to H2/H1 - AND vetoing boosted or resolved

In [ ]:
print(f"Total number of signal events: {len(events)}")
print(f"2 AK8 jets matched to H1 and H2: {ak.sum(boosted)}")
print(f"4 AK4 jets matched to H1 and H2: {ak.sum(resolved)}")
print(f"2 AK4 jets matched to H1/H2 and 1 AK8 jet matched to H2/H1: {ak.sum(semi_resolved)}")
print(f"Outside of these categories: {ak.sum(not_categorized)}")
print(
    f"Sum of all: {ak.sum(not_categorized) + ak.sum(resolved) + ak.sum(boosted) + ak.sum(semi_resolved)}"
)

## Tasks for billy (gen-level)

Understand behavior at gen-level of `not-categorized events` (events that are not boosted, resolved or semi-resolved):
- plot gen MHH for all events (95248 events) and for each of the 4 categories above
- further categorize `not-categorized events`:
  - what is the number of AK4 Jets matched to H1 and matched to H2 (1 + 2, 2 + 1, 0?. >2?)
  - what is the number of AK8 Jets matched to H1 and matched to H2 (1 + 2, 2 + 1, 0?. >2?)


In [ ]:
# e.g. number of ak4 jets matched to h1 and h2 for non-categorized events
print(num_ak4m2h1[not_categorized])
print(num_ak4m2h2[not_categorized])

## Reco-level studies

- Understand FatJet tagger score + particle net mass distributions for events in each gen-level category

In [ ]:
fj_mwh1_idx = events["ak8FatJetHiggsMatchIndex"].to_numpy() == 0
fj_mwh2_idx = events["ak8FatJetHiggsMatchIndex"].to_numpy() == 1

fj_xbb = events["ak8FatJetPNetXbb"].to_numpy()
fj_mass = events["ak8FatJetPNetMass"].to_numpy()

In [ ]:
def plot_h1h2_fj(x, label, bins):
    fj_mwh1_semi_resolved = x[semi_resolved][fj_mwh1_idx[semi_resolved]]
    fj_mwh2_semi_resolved = x[semi_resolved][fj_mwh2_idx[semi_resolved]]

    fj_mwh1_boosted = x[boosted][fj_mwh1_idx[boosted]]
    fj_mwh2_boosted = x[boosted][fj_mwh2_idx[boosted]]

    fj_mwh1_resolved = x[resolved][fj_mwh1_idx[resolved]]
    fj_mwh2_resolved = x[resolved][fj_mwh2_idx[resolved]]

    fj_mwh1_outside = x[not_categorized][fj_mwh1_idx[not_categorized]]
    fj_mwh2_outside = x[not_categorized][fj_mwh2_idx[not_categorized]]

    var_axis = hist.axis.Variable(bins, name="var", label="variable")
    cat_axis = hist.axis.StrCategory([], name="cat", growth=True)

    h1 = hist.Hist(var_axis, cat_axis)
    h1.fill(var=fj_mwh1_boosted, cat="boosted")
    h1.fill(var=fj_mwh1_semi_resolved, cat="semi_resolved")
    h1.fill(var=fj_mwh1_resolved, cat="resolved")
    h1.fill(var=fj_mwh1_outside, cat="non-categorized")

    h2 = hist.Hist(var_axis, cat_axis)
    h2.fill(var=fj_mwh2_boosted, cat="boosted")
    h2.fill(var=fj_mwh2_semi_resolved, cat="semi_resolved")
    h2.fill(var=fj_mwh2_resolved, cat="resolved")
    h2.fill(var=fj_mwh2_outside, cat="non-categorized")

    fig, ax = plt.subplots(1, 2, figsize=(8, 4))
    hep.histplot(h1[{"cat": "boosted"}], ax=ax[0], label="Boosted")
    hep.histplot(h1[{"cat": "resolved"}], ax=ax[0], label="Resolved")
    hep.histplot(h1[{"cat": "semi_resolved"}], ax=ax[0], label="Semi-Resolved")
    hep.histplot(h1[{"cat": "non-categorized"}], ax=ax[0], label="Outside of these categories")
    leg = ax[0].legend(fontsize=10)
    leg.set_title("Gen-Level categories", prop={"size": 10})
    ax[0].set_xlabel(label + " - matched to H1")

    hep.histplot(h2[{"cat": "boosted"}], ax=ax[1], label="Boosted")
    hep.histplot(h2[{"cat": "resolved"}], ax=ax[1], label="Resolved")
    hep.histplot(h2[{"cat": "semi_resolved"}], ax=ax[1], label="Semi-Resolved")
    hep.histplot(h2[{"cat": "non-categorized"}], ax=ax[1], label="Outside of these categories")
    leg = ax[1].legend(fontsize=10)
    leg.set_title("Gen-Level categories", prop={"size": 10})
    ax[1].set_xlabel(label + " matched to H2")

In [ ]:
plot_h1h2_fj(fj_xbb, r"fj $T_{Xbb}$ Score", np.arange(0, 1, 0.03))
plot_h1h2_fj(fj_mass, r"fj Mass", np.arange(0, 200, 7))

## Tasks for billy (reco-level)

Understand behavior at gen-level of `not-categorized events` (events that are not boosted, resolved or semi-resolved):
- 2d plot of fatjet reco pnet mass vs Xbb score for each category
- Plot fatjet reco-mass and Xbb score for events that fail a tight boosted selection:
  - tight boosted selection: (fj_0_Xbb > 0.99) & (fj_0_pnet_mass > 100) & (fj_0_pnet_mass < 140>) & (fj_1_Xbb > 0.99) & (fj_1_pnet_mass > 100) & (fj_1_pnet_mass < 140>) where fj_0 and fj_1 are the fatjets in the event (sorted by Xbb, 0 has the highest Xbb score)
